<a href="https://colab.research.google.com/github/greapis/Tensorflow_Python/blob/main/Tensorflow_AI_LS_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Keras abd Sklearn libraries
    # Importing KerasClassifier Library : Keras vs Sklearn 호환성 이슈 해결

